In [ ]:
import json
from collections import OrderedDict

from flask import Flask, request, abort
from haversine import haversine

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, LocationMessage
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
line_bot_api = LineBotApi(env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
domain = env['YOUR_DOMAIN']


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text))


@handler.add(MessageEvent, message=LocationMessage)
def handle_message(event):    
    sites = OrderedDict([('臺北站', [121.514853, 25.037658]),
                         ('士林站', [121.503019, 25.090253]),
                         ('新莊站', [121.446756, 25.051478]),
                         ('三重站', [121.500625, 25.057373]),
                         ('蘆洲站', [121.472331, 25.086594])])
    
    t = (event.message.latitude, event.message.longitude)
    map_haversine = map(haversine, [t]*len(sites), [e[::-1] for e in list(sites.values())])
    site = sorted(zip(sites, map_haversine), key=lambda x:x[1])[0][0]
    
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(site))

    
if __name__ == "__main__":
    app.run()